In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df['subscription_datetime'] = pd.to_datetime(
    df['subscription_datetime'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

df['sub_year'] = df['subscription_datetime'].dt.year
df['sub_month'] = df['subscription_datetime'].dt.month
df['sub_day'] = df['subscription_datetime'].dt.day
df['sub_dayofweek'] = df['subscription_datetime'].dt.dayofweek
df['sub_hour'] = df['subscription_datetime'].dt.hour

In [ ]:
df.drop(columns=['subscription_datetime'], inplace=True)

In [ ]:
# On sépare la date en jours/mois/heure etc Car, selon année la méthode de paiement peut varier
# Peut-être que selon les mois il y a plus de non-paiement
# Inscription le weekend ou en semaine peut aussi jouer sur le paiement
# L'heure aussi peut jouer un rôle (inscription la nuit ou le jour)

In [ ]:
# On peut imaginer analyse les dates avant, mais attention sauf si on remarque quelque chose avec la date, il vaut mieux la garder pour la prédiction !

# Ce qui est possible pour la date c'est de regarder la date la plus récente qui sera notre référence. On fait la différence de toutes nos dates avec elle pour obtenir l'ancienneté !

In [ ]:
# On transforme les textes en bool (0/1) pour la compréhension du modèle
# Par exemple, channel (web/mobile) devient channel_WEB (0/1)

In [ ]:
df = pd.get_dummies(
    df,
    columns=['channel', 'energy_type', 'ported_phone_number'],
    drop_first=True
)

In [ ]:
# Pour le choses où il y a plus de catégories ex email_domain
# Ce qu'on me propose c'est de transormer les catégories en % d'occurences, par exemple gmail représente 60% de présence, donc on transforme en 60%
# Ce qu'on me propose c'est de transormer les catégories en % d'impayés, exemple sur 50% des adresses gmail sont en impayes, on transforme tous les gmail en 50%

In [ ]:
# Phone plus safe que web (moins d'éléments mais toujours ok), web dans la moyenne

In [ ]:
for col in ['email_domain', 'bank_name', 'zip_code_prefix']:
    freq = df[col].value_counts(normalize=True)
    df[col] = df[col].map(freq)

In [ ]:
carrier_rate = df.groupby('phone_carrier')['top_unpaid'].mean()
df['phone_carrier_enc'] = df['phone_carrier'].map(carrier_rate)

df.drop(columns=['phone_carrier'], inplace=True)

In [ ]:
df

,email_domain,phone_carrier,zip_code_prefix,monthly_amount,bank_name,top_unpaid,sub_year,sub_month,sub_day,sub_dayofweek,sub_hour,channel_WEB,energy_type_GAZ,ported_phone_number_1
0,0.0318,SFR,0.0642,4.0,0.0694,0,2025,1,14,1,16,True,False,True
1,0.0194,SFR,0.0206,6.0,0.0694,0,2025,2,1,5,16,True,False,False
2,0.0240,FREE MOBILE,0.0074,10.0,0.0750,0,2024,8,2,4,13,False,True,True
3,0.0372,BOUYGUES TELECOM,0.0198,4.0,0.0180,0,2024,8,29,3,12,False,True,True
4,0.0392,BOUYGUES TELECOM,0.0170,13.0,0.0416,0,2024,9,26,3,13,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.5778,SFR,0.0286,11.0,0.0214,1,2024,5,23,3,10,False,True,True
4996,0.0002,SFR,0.0022,9.0,0.0858,0,2025,2,24,0,11,False,True,True
4997,0.5778,SFR,0.0166,7.0,0.1184,0,2024,9,26,3,19,True,False,True
4998,0.0004,FREE MOBILE,0.0116,9.0,0.0784,1,2024,8,6,1,16,True,True,True
